In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", RuntimeWarning)

import sys
sys.path.append('../../dstorch/')

import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from dstorch import preprocessing, draw, transforms, predict, postprocessing, metrics, utils, io
import cv2
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import pydash as _

In [4]:
TEST_PATH = '../data/stage1_test/'
test_ids, _ = io.read_test_data(TEST_PATH, d=3)

test_images = []
for _id in tqdm(test_ids):
    test_images.append(cv2.imread("../output_data/without_scale_05/test/{}.png".format(_id)))

100%|██████████| 65/65 [00:00<00:00, 322.15it/s]


In [5]:
def get_predictions(model_path, ids, images, path_images, resize):
    transform = transforms.DualCompose(
        [
            transforms.ImageOnly(transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]))
        ]
    )
    model = torch.load(model_path)
    predictions = predict.predict(model, ids, path_images, transform, 64, 2, 1)

    if resize:
        predictions = [cv2.resize(x, (y.shape[1], y.shape[0])) for x, y in zip(predictions, images)]

    return predictions

In [6]:
def weighted_sum(predictions, weights):
    return np.sum(np.stack(predictions) * np.array(weights)[..., np.newaxis, np.newaxis, np.newaxis], axis=0) / np.sum(weights)


def make_ensemble(weights, *all_predictions):
    return [weighted_sum(predictions, weights) for predictions in zip(*all_predictions)]

In [7]:
without_scale_05_489_predictions = get_predictions(
    'models/TernausNet34_without_scale_05/local_489.pth', test_ids, 
    test_images, '../output_data/without_scale_05/scaled_test/{}.png', True
)

without_scale_05_389_predictions = get_predictions(
    'models/TernausNet34_without_scale_05/local_389.pth', test_ids, 
    test_images, '../output_data/without_scale_05/scaled_test/{}.png', True
)

without_scale_05_289_predictions = get_predictions(
    'models/TernausNet34_without_scale_05/local_289.pth', test_ids, 
    test_images, '../output_data/without_scale_05/scaled_test/{}.png', True
)

Predict: 100%|██████████| 65/65 [00:03<00:00, 20.56it/s]


In [8]:
test_predictions = make_ensemble(
    [0.2, 0.2, 0.2], 
    with_scale_05_489_predictions, 
    with_scale_05_389_predictions, 
    with_scale_05_289_predictions
)

In [18]:
new_ids, rles = utils.mask2rle(test_predictions, test_ids, lambda x: postprocessing.watershed(
        x, mask_threshold=0.5, center_threshold=0.5, padding_size=5, min_size=25)
)

100%|██████████| 65/65 [00:01<00:00, 34.18it/s]


In [19]:
from datetime import datetime
sub = utils.make_submission(
    '../output_data/test_predict_labels.csv'.format(datetime.now()), new_ids, rles
)